In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

In [2]:
# Load the loan dataset
X_train = pd.read_csv("raw_data\X_train.csv")

In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1199861 entries, 0 to 1199860
Data columns (total 45 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   index                       1199861 non-null  int64  
 1   acc_now_delinq              1199861 non-null  float64
 2   addr_state                  1199861 non-null  object 
 3   annual_inc                  1199861 non-null  float64
 4   chargeoff_within_12_mths    1199824 non-null  float64
 5   collections_12_mths_ex_med  1199824 non-null  float64
 6   delinq_2yrs                 1199861 non-null  float64
 7   dti                         1199861 non-null  float64
 8   earliest_cr_line            1199861 non-null  object 
 9   emp_length                  1128114 non-null  object 
 10  fico_range_high             1199861 non-null  float64
 11  fico_range_low              1199861 non-null  float64
 12  funded_amnt                 1199861 non-null  float64
 1

In [15]:
# condition state = connecticut
condition = (X_train['addr_state'] == 'CT') & (X_train['zip_code'])
connecticut = X_train.loc[condition]
connecticut = connecticut.loc[:, ['addr_state', 'zip_code']]
connecticut

,addr_state,zip_code
98,CT,064xx
193,CT,060xx
218,CT,064xx
277,CT,067xx
324,CT,060xx
...,...,...
1199284,CT,064xx
1199364,CT,063xx
1199390,CT,060xx
1199451,CT,060xx


In [17]:
X_train['purpose'].value_counts() #.unique() #.nunique()

debt_consolidation    684522
credit_card           268419
home_improvement       78972
other                  73218
major_purchase         26885
medical                14327
small_business         13462
car                    13214
vacation                8686
moving                  8423
house                   7143
wedding                 1555
renewable_energy         827
educational              208
Name: purpose, dtype: int64

In [18]:
X_train.corr()

,index,acc_now_delinq,annual_inc,chargeoff_within_12_mths,collections_12_mths_ex_med,delinq_2yrs,dti,fico_range_high,fico_range_low,funded_amnt,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,open_acc,open_il_24m,open_rv_24m,percent_bc_gt_75,pub_rec,pub_rec_bankruptcies,tax_liens,total_acc
index,1.000000,0.000937,0.000753,-0.000880,0.000417,0.002134,-0.000775,-0.001722,-0.001723,0.000218,...,0.001102,-0.000043,0.001166,0.001154,0.001656,0.000655,0.000249,-0.000603,0.001093,0.000078
acc_now_delinq,0.000937,1.000000,0.005950,0.035715,0.013021,0.118874,0.004890,-0.039446,-0.039446,0.001224,...,0.052830,-0.006497,0.018514,-0.007412,-0.003624,-0.017659,0.000329,-0.009722,0.007829,0.025790
annual_inc,0.000753,0.005950,1.000000,0.003787,-0.003220,0.020794,-0.089276,0.036357,0.036358,0.169481,...,0.003164,0.027134,0.072762,0.049260,-0.006135,-0.006069,-0.003612,-0.023770,0.019584,0.092520
chargeoff_within_12_mths,-0.000880,0.035715,0.003787,1.000000,0.039861,0.144410,-0.002343,-0.053685,-0.053686,-0.002102,...,0.216124,0.001849,0.006941,-0.000527,0.004432,-0.010978,-0.003795,-0.010437,-0.000927,0.041557
collections_12_mths_ex_med,0.000417,0.013021,-0.003220,0.039861,1.000000,0.068307,-0.004601,-0.071823,-0.071824,-0.019071,...,0.100378,0.012266,0.008088,-0.007490,0.018646,-0.030928,0.012264,-0.002214,0.012032,0.004388
delinq_2yrs,0.002134,0.118874,0.020794,0.144410,0.068307,1.000000,-0.012259,-0.176767,-0.176769,-0.003309,...,0.658353,-0.025911,0.050699,-0.027463,-0.046214,-0.004675,-0.021959,-0.049365,0.011012,0.117736
dti,-0.000775,0.004890,-0.089276,-0.002343,-0.004601,-0.012259,1.000000,-0.072275,-0.072272,0.021245,...,-0.017793,0.083563,0.299783,0.257680,0.028099,0.168933,-0.040490,-0.024900,-0.029085,0.238391
fico_range_high,-0.001722,-0.039446,0.036357,-0.053685,-0.071823,-0.176767,-0.072275,1.000000,1.000000,0.095587,...,-0.103244,-0.084047,0.022428,0.006330,-0.127526,-0.418789,-0.190066,-0.197647,-0.059441,0.025295
fico_range_low,-0.001723,-0.039446,0.036358,-0.053686,-0.071824,-0.176769,-0.072272,1.000000,1.000000,0.095588,...,-0.103245,-0.084044,0.022429,0.006333,-0.127525,-0.418793,-0.190069,-0.197649,-0.059441,0.025296
funded_amnt,0.000218,0.001224,0.169481,-0.002102,-0.019071,-0.003309,0.021245,0.095587,0.095588,1.000000,...,-0.020730,-0.019437,0.186427,0.029351,-0.015241,0.024636,-0.059670,-0.087055,0.009023,0.209557


In [19]:
print(X_train.columns)

Index(['index', 'acc_now_delinq', 'addr_state', 'annual_inc',
       'chargeoff_within_12_mths', 'collections_12_mths_ex_med', 'delinq_2yrs',
       'dti', 'earliest_cr_line', 'emp_length', 'fico_range_high',
       'fico_range_low', 'funded_amnt', 'home_ownership', 'inq_last_12m',
       'installment', 'int_rate', 'issue_d', 'loan_amnt', 'mort_acc',
       'mths_since_last_delinq', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl',
       'num_rev_accts', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd',
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'open_acc', 'open_il_24m',
       'open_rv_24m', 'percent_bc_gt_75', 'pub_rec', 'pub_rec_bankruptcies',
       'purpose', 'revol_util', 'tax_liens', 'term', 'title', 'total_acc',
       'verification_status', 'zip_code'],
      dtype='object')


In [20]:
X_train.head()

,index,acc_now_delinq,addr_state,annual_inc,chargeoff_within_12_mths,collections_12_mths_ex_med,delinq_2yrs,dti,earliest_cr_line,emp_length,...,pub_rec,pub_rec_bankruptcies,purpose,revol_util,tax_liens,term,title,total_acc,verification_status,zip_code
0,0,0.0,UT,80000.0,0.0,0.0,0.0,30.49,Oct-1996,4 years,...,1.0,1.0,credit_card,56.1%,0.0,36 months,Credit card refinancing,26.0,Source Verified,847xx
1,1,0.0,CA,82000.0,0.0,0.0,0.0,7.00,Mar-1993,10+ years,...,1.0,1.0,credit_card,88.3%,0.0,36 months,Credit card refinancing,11.0,Not Verified,900xx
2,2,0.0,NV,46080.0,0.0,0.0,1.0,17.32,Nov-2000,3 years,...,0.0,0.0,credit_card,18.1%,0.0,36 months,Credit card refinancing,31.0,Source Verified,895xx
3,3,0.0,AZ,30000.0,0.0,0.0,0.0,7.80,Jan-2010,< 1 year,...,0.0,0.0,car,33.4%,0.0,36 months,Car financing,24.0,Source Verified,853xx
4,4,0.0,OH,70000.0,0.0,0.0,0.0,13.36,Jan-2002,< 1 year,...,0.0,0.0,other,88.6%,0.0,60 months,Other,16.0,Not Verified,453xx


#### find boolean columns

In [22]:
# find out which column in a pandas dataframe has two distinct values
def get_column_with_two_distinct_values(df: pd.DataFrame) -> str:
    """
    Returns the name of the column that has only two distinct values.
    """
    return df.columns[df.nunique() == 2][0]

In [23]:
print(get_column_with_two_distinct_values(X_train))

term


In [24]:
X_train['term'].value_counts() #.nunique

 36 months    903699
 60 months    296162
Name: term, dtype: int64

In [25]:
# num_cols = ['annual_inc', 'dti', 'fico_range_high', 'fico_range_low', 'loan_amnt',  'revol_util']

In [26]:
# X_train[num_cols]

In [27]:
if X_train.dtypes.all() == bool:
    print('The DataFrame is of boolean data type')

In [28]:
print('The data types of X_train are:\n', X_train.dtypes)

The data types of X_train are:
 index                           int64
acc_now_delinq                float64
addr_state                     object
annual_inc                    float64
chargeoff_within_12_mths      float64
collections_12_mths_ex_med    float64
delinq_2yrs                   float64
dti                           float64
earliest_cr_line               object
emp_length                     object
fico_range_high               float64
fico_range_low                float64
funded_amnt                   float64
home_ownership                 object
inq_last_12m                  float64
installment                   float64
int_rate                       object
issue_d                        object
loan_amnt                     float64
mort_acc                      float64
mths_since_last_delinq        float64
mths_since_recent_bc_dlq      float64
mths_since_recent_inq         float64
num_accts_ever_120_pd         float64
num_actv_bc_tl                float64
num_rev_accts     

In [30]:
# select only columns with numeric data types
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
numeric_cols
# or 
# df.select_dtypes(include=[np.number])

Index(['index', 'acc_now_delinq', 'annual_inc', 'chargeoff_within_12_mths',
       'collections_12_mths_ex_med', 'delinq_2yrs', 'dti', 'fico_range_high',
       'fico_range_low', 'funded_amnt', 'inq_last_12m', 'installment',
       'loan_amnt', 'mort_acc', 'mths_since_last_delinq',
       'mths_since_recent_bc_dlq', 'mths_since_recent_inq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_rev_accts', 'num_sats',
       'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m',
       'num_tl_op_past_12m', 'open_acc', 'open_il_24m', 'open_rv_24m',
       'percent_bc_gt_75', 'pub_rec', 'pub_rec_bankruptcies', 'tax_liens',
       'total_acc'],
      dtype='object')

In [31]:
non_numeric_cols = X_train.select_dtypes(exclude='number').columns
non_numeric_cols

Index(['addr_state', 'earliest_cr_line', 'emp_length', 'home_ownership',
       'int_rate', 'issue_d', 'purpose', 'revol_util', 'term', 'title',
       'verification_status', 'zip_code'],
      dtype='object')

In [32]:
non_numeric_cols.value_counts()

addr_state             1
earliest_cr_line       1
emp_length             1
home_ownership         1
int_rate               1
issue_d                1
purpose                1
revol_util             1
term                   1
title                  1
verification_status    1
zip_code               1
dtype: int64

In [33]:
X_train[non_numeric_cols]

,addr_state,earliest_cr_line,emp_length,home_ownership,int_rate,issue_d,purpose,revol_util,term,title,verification_status,zip_code
0,UT,Oct-1996,4 years,RENT,12.99%,Apr-2016,credit_card,56.1%,36 months,Credit card refinancing,Source Verified,847xx
1,CA,Mar-1993,10+ years,MORTGAGE,10.42%,Sep-2017,credit_card,88.3%,36 months,Credit card refinancing,Not Verified,900xx
2,NV,Nov-2000,3 years,MORTGAGE,8.99%,Aug-2016,credit_card,18.1%,36 months,Credit card refinancing,Source Verified,895xx
3,AZ,Jan-2010,< 1 year,OWN,11.53%,Aug-2015,car,33.4%,36 months,Car financing,Source Verified,853xx
4,OH,Jan-2002,< 1 year,RENT,26.30%,Sep-2017,other,88.6%,60 months,Other,Not Verified,453xx
...,...,...,...,...,...,...,...,...,...,...,...,...
1199856,IL,Feb-2002,4 years,RENT,15.31%,Jun-2012,debt_consolidation,82.1%,36 months,debt consolidation,Source Verified,604xx
1199857,NY,Jan-2002,3 years,RENT,6.03%,Nov-2013,credit_card,82.3%,36 months,Credit cards financing,Verified,106xx
1199858,MA,Nov-2002,8 years,OWN,11.99%,Dec-2015,home_improvement,6%,60 months,Home improvement,Source Verified,021xx
1199859,OH,Nov-1991,5 years,MORTGAGE,22.78%,Jun-2012,debt_consolidation,76.2%,60 months,Debt,Verified,454xx


In [34]:
#count unique values in non_numeric_cols each column
print(X_train[non_numeric_cols].nunique())

addr_state                51
earliest_cr_line         748
emp_length                11
home_ownership             6
int_rate                 669
issue_d                  160
purpose                   14
revol_util              1316
term                       2
title                  43858
verification_status        3
zip_code                 936
dtype: int64


In [35]:
#count unique values in non_numeric_cols each column
print(X_train.nunique())

index                         1199861
acc_now_delinq                      7
addr_state                         51
annual_inc                      59051
chargeoff_within_12_mths           11
collections_12_mths_ex_med         13
delinq_2yrs                        32
dti                              4747
earliest_cr_line                  748
emp_length                         11
fico_range_high                    38
fico_range_low                     38
funded_amnt                      1562
home_ownership                      6
inq_last_12m                       45
installment                     81217
int_rate                          669
issue_d                           160
loan_amnt                        1562
mort_acc                           39
mths_since_last_delinq            167
mths_since_recent_bc_dlq          168
mths_since_recent_inq              26
num_accts_ever_120_pd              43
num_actv_bc_tl                     36
num_rev_accts                     111
num_sats    

In [36]:
max_interest_rate = X_train['int_rate'].max()
max_interest_rate

' 30.99%'

In [39]:
max_interest_rate = float(max_interest_rate.replace(" ", "").replace("%", "")) #.astype('int')
max_interest_rate

30.99

In [40]:
X_train['mths_since_recent_bc_dlq'].max() #.nunique() #.unique() #

198.0

#### Categorical Variables

In [41]:
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == object]
# or
# categorical_cols = X_train.select_dtypes(include=['object','category'])
categorical_cols

['addr_state',
 'earliest_cr_line',
 'emp_length',
 'home_ownership',
 'int_rate',
 'issue_d',
 'purpose',
 'revol_util',
 'term',
 'title',
 'verification_status',
 'zip_code']

###### Numerical Variables

In [44]:
numerical_cols = X_train.select_dtypes(include='number').columns
numerical_cols

Index(['index', 'acc_now_delinq', 'annual_inc', 'chargeoff_within_12_mths',
       'collections_12_mths_ex_med', 'delinq_2yrs', 'dti', 'fico_range_high',
       'fico_range_low', 'funded_amnt', 'inq_last_12m', 'installment',
       'loan_amnt', 'mort_acc', 'mths_since_last_delinq',
       'mths_since_recent_bc_dlq', 'mths_since_recent_inq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_rev_accts', 'num_sats',
       'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m',
       'num_tl_op_past_12m', 'open_acc', 'open_il_24m', 'open_rv_24m',
       'percent_bc_gt_75', 'pub_rec', 'pub_rec_bankruptcies', 'tax_liens',
       'total_acc'],
      dtype='object')

In [45]:
# term mapping
X_train.term.unique()

array([' 36 months', ' 60 months'], dtype=object)

In [46]:
term_values = {' 36 months': 36, ' 60 months': 60}
X_train['term'] = X_train.term.map(term_values)

In [47]:
X_train.term.unique()

array([36, 60], dtype=int64)

In [48]:
X_train.zip_code.value_counts()

945xx    12979
750xx    12823
112xx    12605
606xx    11289
300xx    11150
         ...  
513xx        1
528xx        1
521xx        1
205xx        1
516xx        1
Name: zip_code, Length: 936, dtype: int64

#### Check for duplicates columns & features

In [50]:
# Remove duplicate Features
# X_train = X_train.T.drop_duplicates()
# df = df.loc[:,~df.columns.duplicated()].copy()
X_train = X_train.loc[:,~X_train.columns.duplicated()].copy()

## Train Test Split

#### Note: there is no target variable

## Removing Outliers

In [53]:
# print(X_train[X_train['dti'] <= 50].shape)
# print(X_train.shape)

In [54]:
X_train['dti'].unique()

array([30.49,  7.  , 17.32, ..., 43.3 , 49.03, 44.41])

In [55]:
X_train['dti'].nunique()

4747

In [56]:
X_train[X_train['dti'] <= 50].shape

(1199861, 45)

##### find duplicate rows across all columns

In [57]:
duplicateRows = X_train[X_train.duplicated()]
duplicateRows

,index,acc_now_delinq,addr_state,annual_inc,chargeoff_within_12_mths,collections_12_mths_ex_med,delinq_2yrs,dti,earliest_cr_line,emp_length,...,pub_rec,pub_rec_bankruptcies,purpose,revol_util,tax_liens,term,title,total_acc,verification_status,zip_code


In [58]:
# Remove Duplicate Rows
X_train.drop_duplicates(inplace=True)